<a href="https://colab.research.google.com/github/damianwgriggs/The-Griggs-Cold-Chain/blob/main/griggschainexperiment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# ==========================================
# MASTER CELL: PROTOCOL V4 (SMART CPMG)
# ==========================================

!pip install -q qiskit qiskit-ibm-runtime qiskit-aer numpy
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# --- HELPER: CPMG SEQUENCE (The Industrial Standard) ---
def apply_cpmg_sequence(qc, qubit, duration_us, n_pulses=4):
    """
    Applies CPMG-N (N X-gates).
    Structure: [d/2] - X - [d] - X - [d] - X ... - [d/2]
    Robust against pulse angle errors and T2 dephasing.
    """
    if duration_us <= 0:
        return

    # Calculate delay intervals
    total_intervals = 2 * n_pulses # N pulses create N+1 gaps, but ends are half-gaps
    # Actually, standard CPMG definition:
    # 4 pulses: d/2 - X - d - X - d - X - d - X - d/2
    # Total time = 4d.

    dt_delay = duration_us / n_pulses # The length of one full interval
    d_edge = dt_delay / 2.0

    # Sequence
    qc.delay(d_edge, qubit, unit='us')
    for _ in range(n_pulses):
        qc.x(qubit)
        if _ < n_pulses - 1: # Don't delay after the last pulse yet
            qc.delay(dt_delay, qubit, unit='us')
    qc.delay(d_edge, qubit, unit='us')

# --- PART 1: THE CIRCUIT ---
def create_smart_freezer_circuit(travel_time_us=10, hold_time_us=20):
    qr = QuantumRegister(3, 'q')
    cr_hop1 = ClassicalRegister(2, 'hop1')
    cr_hop2 = ClassicalRegister(2, 'hop2')
    cr_final = ClassicalRegister(1, 'result')
    qc = QuantumCircuit(qr, cr_hop1, cr_hop2, cr_final)

    theta = 2 * np.arccos(np.sqrt(0.75))

    # --- PHASE 0: THE "WARM" WAIT (Optimized) ---
    # We intentionally DO NOT pulse q2 here.
    # q2 is in |0>, so it is immune to dephasing.
    # Pulsing it would only expose it to T1 decay.

    qc.ry(theta, 0) # Load Cargo
    if travel_time_us > 0:
        qc.delay(travel_time_us, 0, unit='us')
        # q2 just waits (implicitly) or we can explicit delay
        qc.delay(travel_time_us, 2, unit='us')

    qc.barrier()

    # --- PHASE 1: DELIVERY ---
    qc.h(1)
    qc.cx(1, 2)
    qc.cx(0, 1)
    qc.h(0)
    qc.measure(0, cr_hop1[0])
    qc.measure(1, cr_hop1[1])

    with qc.if_test((cr_hop1[1], 1)):
        qc.x(2)
    with qc.if_test((cr_hop1[0], 1)):
        qc.z(2)
    qc.barrier()

    # --- PHASE 2: THE FREEZER (CPMG) ---
    qc.reset(0)
    qc.reset(1)

    # NOW we turn on the freezer because q2 holds data.
    if hold_time_us > 0:
        # CPMG-4 (4 X pulses) is a sweet spot for 20us on Torino
        apply_cpmg_sequence(qc, 2, hold_time_us, n_pulses=4)

    qc.barrier()

    # --- PHASE 3: RETURN & CHECK ---
    qc.h(1)
    qc.cx(1, 0)
    qc.cx(2, 1)
    qc.h(2)
    qc.measure(2, cr_hop2[0])
    qc.measure(1, cr_hop2[1])

    with qc.if_test((cr_hop2[1], 1)):
        qc.x(0)
    with qc.if_test((cr_hop2[0], 1)):
        qc.z(0)

    # Phase Verification (Inverse)
    qc.ry(-theta, 0)
    qc.measure(0, cr_final)

    return qc

# --- PART 2: DEPLOYMENT ---
MY_TOKEN = "MYTOKENWAZHERE"

if MY_TOKEN == "YOUR_TOKEN_HERE" or MY_TOKEN == "":
    print("STOP: Please paste your IBM Token.")
else:
    try:
        service = QiskitRuntimeService(channel="ibm_quantum_platform", token=MY_TOKEN)

        print("[Step 1] Initializing Smart CPMG Protocol...")
        backend = service.least_busy(min_num_qubits=127, operational=True, simulator=False)
        print(f"         Targeting: {backend.name}")

        print("[Step 2] Compiling V4 Circuit (Just-In-Time CPMG)...")
        qc_v4 = create_smart_freezer_circuit(travel_time_us=10, hold_time_us=20)

        pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
        isa_qc = pm.run(qc_v4)

        print("[Step 3] Executing...")
        sampler = Sampler(backend)
        job = sampler.run([isa_qc], shots=8192)
        print(f"         Job ID: {job.job_id()}")

        result = job.result()
        counts = result[0].data.result.get_counts()

        zeros = 0
        total = 0
        for k, v in counts.items():
            if k.startswith('0'):
                zeros += v
            total += v

        fidelity = (zeros/total)*100

        print("\n" + "*"*45)
        print(f" DAMIAN GRIGGS: V4 SMART FREEZER")
        print("*"*45)
        print(f"Sequence:       CPMG-4")
        print(f"Phase 0 (Idle): OFF (Saved T1)")
        print(f"Phase 2 (Hold): ON (CPMG Protected)")
        print("-" * 45)
        print(f"ACTUAL SCORE:   {fidelity:.2f}%")
        print("*"*45)

        if fidelity > 75.0:
            print("❄️ SUCCESS: Smart management beat the brute force.")
        elif fidelity > 64.69:
            print("📈 RECOVERY: We are back on the right track.")
        else:
            print("🤔 NOTE: Hardware noise floor reached.")

    except Exception as e:
        print(f"Error: {e}")

qiskit_runtime_service._discover_account:WARNING:2025-12-19 15:57:43,852: Loading account with the given token. A saved account will not be used.
qiskit_runtime_service.__init__:WARNING:2025-12-19 15:57:45,910: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().


[Step 1] Initializing Smart CPMG Protocol...


qiskit_runtime_service.backends:WARNING:2025-12-19 15:57:46,396: Loading instance: open-instance, plan: open
qiskit_runtime_service.backends:WARNING:2025-12-19 15:57:47,944: Using instance: open-instance, plan: open


         Targeting: ibm_torino
[Step 2] Compiling V4 Circuit (Just-In-Time CPMG)...
[Step 3] Executing...
         Job ID: d52nbv7p3tbc73alnpt0

*********************************************
 DAMIAN GRIGGS: V4 SMART FREEZER
*********************************************
Sequence:       CPMG-4
Phase 0 (Idle): OFF (Saved T1)
Phase 2 (Hold): ON (CPMG Protected)
---------------------------------------------
ACTUAL SCORE:   72.52%
*********************************************
📈 RECOVERY: We are back on the right track.
